# PF Cash Flow Projections: Q3-Q4 '23
#### Adrafül Labs

**Notes:**

Cash Flows will be projected using a combination of:
- total revenue, which can be found in `sales.ipynb`
- total expenses, which can be found in `expenses.ipynb`
- inventory purchases, which can be found in `inventory.ipynb`
- sg&a cash outflows

**The primary objective with this notebook will be to ensure sufficient initial capital levels to sustain our high-starting cost inventory purchases and increasing marketing budget.**

This case assumes a capital raise amount of 250,000.

Charts will be produced using the plotly charting library.

All future projections are estimates.

_________

We begin by bringing in all local data that will be used to forecast cash flows.

In [1]:
# imports
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [2]:
# import sales data
sales_df = pd.read_excel('../../data/outputs/projected_daily_sales.xlsx')
sales_df.drop('Unnamed: 0', axis=1, inplace=True)
# import expenses data
expenses_df = pd.read_excel('../../data/outputs/projected_daily_expenses.xlsx')
expenses_df.drop('Unnamed: 0', axis=1, inplace=True)
# import inventory purchases
inventory_purchases_df = pd.read_excel('../../data/imports/excel/inventory_projected_purchases.xlsx')
# import sg&a outflows
sga_outflows = pd.read_excel('../../data/imports/excel/sga_projected_expenses.xlsx')

In [3]:
STARTING_CASH = 32000
RAISE_AMOUNT = 250000
STARTING_CASH += RAISE_AMOUNT

In [4]:
projected_cash_flows = pd.DataFrame(columns=['date', 'cash_in', 'cash_out', 'cash_balance'])
projected_cash_flows['date'] = pd.date_range(start='2023-08-01', end='2023-12-31')
projected_cash_flows['cash_in'] = 0
projected_cash_flows['cash_out'] = 0
projected_cash_flows['cash_balance'] = STARTING_CASH
projected_cash_flows.head()

,date,cash_in,cash_out,cash_balance
0,2023-08-01,0,0,282000
1,2023-08-02,0,0,282000
2,2023-08-03,0,0,282000
3,2023-08-04,0,0,282000
4,2023-08-05,0,0,282000


**Cash Inflows**

We will first add cash inflows as a result of sales.

In [5]:
projected_cash_flows['cash_in'] = sales_df['total_gross_sales']
projected_cash_flows.head()

,date,cash_in,cash_out,cash_balance
0,2023-08-01,1741.344874,0,282000
1,2023-08-02,238.777757,0,282000
2,2023-08-03,1292.313418,0,282000
3,2023-08-04,177.107556,0,282000
4,2023-08-05,150.000000,0,282000


**Cash Outflows :(**

We will now update cash outflows as a result of marketing, shipping loss, sg&a, and inventory purchases.

In [6]:
projected_cash_flows['cash_out'] += expenses_df['marketing'] + expenses_df['shipping_loss']
projected_cash_flows['date'] = pd.to_datetime(projected_cash_flows['date'])
for row in sga_outflows.itertuples():
    date = pd.to_datetime(row.date)
    projected_cash_flows.loc[projected_cash_flows['date'] == date, 'cash_out'] += row.sga_amount
for row in inventory_purchases_df.itertuples():
    date = pd.to_datetime(row.date)
    projected_cash_flows.loc[projected_cash_flows['date'] == date, 'cash_out'] += row.purchase_amount
# Update balance
last_balance = STARTING_CASH

for row in projected_cash_flows.itertuples():
    balance = last_balance
    balance += row.cash_in
    balance -= row.cash_out
    projected_cash_flows.loc[row.Index, 'cash_balance'] = balance
    last_balance = balance

projected_cash_flows.head(10)

,date,cash_in,cash_out,cash_balance
0,2023-08-01,1741.344874,20488.093136,263253.251738
1,2023-08-02,238.777757,24.745751,263467.283744
2,2023-08-03,1292.313418,68.601633,264690.995528
3,2023-08-04,177.107556,2421.497168,262446.605916
4,2023-08-05,150.000000,19.872876,262576.733041
5,2023-08-06,150.000000,19.872876,262706.860165
6,2023-08-07,495.177516,34.491503,263167.546178
7,2023-08-08,939.934595,53.983006,264053.497767
8,2023-08-09,485.061132,34.491503,264504.067395
9,2023-08-10,150.000000,19.872876,264634.194520


Below you can see the projected cash flows over the next 2Q.

In [7]:
# Plot inflows and outflows
pcf_fig = go.Figure(data=go.Scatter(x=projected_cash_flows['date'], y=projected_cash_flows['cash_in'], name='Cash Inflows'))
pcf_fig.add_trace(go.Scatter(x=projected_cash_flows['date'], y=projected_cash_flows['cash_out'], name='Cash Outflows'))
# pcf_fig.add_trace(go.Scatter(x=projected_cash_flows['date'], y=projected_cash_flows['cash_balance'], name='Cash Balance'))
pcf_fig.update_layout(title='Projected Cash Flows', xaxis_title='Date', yaxis_title='Cash Flow ($)')
pcf_fig.show()

Below you can see the projected cash balance over the next 2Q.

In [8]:
# Plot cash balance
balance_fig = go.Figure(data=go.Scatter(x=projected_cash_flows['date'], y=projected_cash_flows['cash_balance'], name='Balance'))
balance_fig.add_trace(go.Scatter(x=projected_cash_flows['date'], y=[0]*len(projected_cash_flows['date']), mode='lines', name='Zero', line=dict(color='red', dash='dash')))
balance_fig.add_trace(go.Scatter(x=projected_cash_flows['date'], y=[STARTING_CASH]*len(projected_cash_flows['date']), mode='lines', name='CF+', line=dict(color='green', dash='dash')))
balance_fig.update_layout(title='Projected Cash Balance', xaxis_title='Date', yaxis_title='Cash Balance ($)')
balance_fig.show()

Above you can see our projected cash balance over time. Due to high capital requirements for inventory purchases, we do not project consistent cash-flow positivity until Q1 2024. Around October - November is when we expect to begin nearing cash-flow positive operations, but the still increasing purchase amounts for inventory will continue to pre

In [14]:
import plotly.express as px
pcf_fig.write_html("../data/outputs/test.html")